<a href="https://colab.research.google.com/github/TiFcode/TiFai/blob/main/TiFai_tif3_object_recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('*** START ********************************************************')

!pwd
!ls

outputDetails = False
outputInfo = True
size = (224, 224)
googleDriveFolderPath = '/content/drive/My Drive/Personal/Dev/ChatGPT/2023-01-05 - AI - object recognition/'

In [ ]:
from google.colab.patches import cv2_imshow

def display_image(image, description):
  print(f'{description}:')
  print(f'Image size: {image.shape}')
  cv2_imshow(image)


In [ ]:
import inspect

def retrieve_variable_name_from_local(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]
    
def retrieve_variable_name_from_call(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

def textForVar(variable):
  return f'{retrieve_variable_name_from_call(variable)} is [{variable}]'

def textForVarWithDesc(variable, description):
  return f'{description} is [{variable}]'

def printInfo(text):
  if outputInfo:
    print(text)

In [ ]:
from google.colab import drive
import glob

# Mount Google Drive
drive.mount('/content/drive')
!pwd
!ls

# Search for a file by name
#pattern = '/content/drive/My Drive/**/*20230105_151957.jpg'
#filenames = glob.glob(pattern, recursive=True)
#print(filenames)


In [ ]:
import os
import cv2

def loadAndProcessAllFilesFromFolder(folderPath):
  filenames = os.listdir(folderPath)

  imagesArray = []
  for filename in filenames:
    filenameFullPath = folderPath + filename
    if outputDetails:
      print(f'Found file [{filename}] with size [{os.stat(filenameFullPath).st_size}] bytes.')
    
    image = cv2.imread(filenameFullPath)
    if outputDetails:
      display_image(image, filenameFullPath)
    imagesArray.append(image)

    if outputDetails:
      print(f'imagesArray contains [{len(imagesArray)}] elements.')

  # Resize the images to a fixed size
  imagesArray = [cv2.resize(image, size) for image in imagesArray]

  # Convert the images to a format that can be used as input to a CNN
  imagesArray = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in imagesArray]

  return imagesArray

In [ ]:
def load_images(folderName):
  print(f'Loading {folderName}...')
  images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + folderName + '/')
  print(f'{folderName} contains [{len(images)}] images.')
  return images

training_images = load_images('training_images')
validation_images = load_images('validation_images')
search_images = load_images('search_images')

print('*** FINISH LOADING IMAGES ********************************************************')


In [ ]:
#from sklearn.model_selection import train_test_split

# Split the object images into a training set and a validation set
#X_train, X_val = train_test_split(object_images, test_size=0.2, random_state=42)


In [ ]:
import numpy as np

def prepare_array(list_of_images):
  print(f'START: Preparing array [{retrieve_variable_name_from_call(list_of_images)}] ...')

  # Convert the list to numpy array
  printInfo(textForVarWithDesc(type(list_of_images), "type(list_of_images)"))
  printInfo(textForVarWithDesc(len(list_of_images), "len(list_of_images)"))
  array_of_images = np.array(list_of_images)
  printInfo(textForVarWithDesc(type(array_of_images), "type(array_of_images)"))
  printInfo(textForVarWithDesc(array_of_images.shape[0], "array_of_images.shape[0]"))

  # Reshape the array
  printInfo(textForVarWithDesc(array_of_images.shape, "array_of_images.shape"))
  #batch_size = array_of_images.shape[0]
  #printInfo(textForVar(batch_size))
  #array_of_images = np.reshape(array_of_images, (batch_size, 224, 224, 3))
  #printInfo(textForVarWithDesc(array_of_images.shape, "array_of_images.shape"))

  print(f'END: Preparing array [{retrieve_variable_name_from_call(list_of_images)}].')

  return array_of_images


# Preparing the image arrays
training_images = prepare_array(training_images)
validation_images = prepare_array(validation_images)
search_images = prepare_array(search_images)

# Create the target data
target_data_yes = np.ones(training_images.shape[0])
target_data_no = np.zeros(training_images.shape[0])
target_data = target_data_yes

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
training_batch_size = training_images.shape[0]
history = model.fit(training_images, target_data, batch_size=training_batch_size, epochs=10, validation_data=(validation_images,))


In [ ]:
# Generate predictions for the search images
predictions = model.predict(search_images)

# Set the threshold
threshold = 0.5

# Loop through the predictions
for i in range(len(predictions)):
  cv2_imshow(search_images[i])
  print(textForVarWithDesc(predictions[i], 'predictions[i]'))
  # If the prediction is greater than the threshold, print "Object found"
  if predictions[i] > threshold:
    print(f"=> Object found in search image {i}\n\n\n")
  # Otherwise, print "Object not found"
  else:
    print(f"Object not found in search image {i}\n\n\n")
